In [25]:
import grpc
import os
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [26]:
host = os.getenv('TF_SERVING_HOST', 'localhost:8500')

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [27]:
from keras_image_helper import create_preprocessor

In [28]:
preprocessor = create_preprocessor('xception', target_size=(150, 150))

In [29]:
url = 'https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/31OEvRveV3L.jpg'
X = preprocessor.from_url(url)

In [30]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [31]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'kitchen-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_24'].CopyFrom(np_to_protobuf(X))

In [32]:
pb_response = stub.Predict(pb_request, timeout=50.0)

In [33]:
preds = pb_response.outputs['dense_20'].float_val

In [34]:
classes = [
    'cup',
    'fork',
    'glass',
    'knife',
    'plate',
    'spoon'
]

In [35]:
dict(zip(classes, preds))

{'cup': 3.666372060775757,
 'fork': -4.526908874511719,
 'glass': 6.12653112411499,
 'knife': -3.056701183319092,
 'plate': -3.772914171218872,
 'spoon': -1.1649322509765625}